# 6.2 特徴選択および特徴量の重要度

モデルの予測に寄与しない特徴を判定することで、それらを取り除き精度向上や計算時間の短縮が可能となる。

ここでは、以下の 3 つの方法を用いてそのような特徴量の判定方法を説明する。

1. 単変量統計を用いる方法
2. 特徴量の重要度を用いる方法
3. 反復して探索する方法

## 6.2.1 単変量統計を用いる方法

各特徴量と目的変数から何らかの統計量を計算し、その統計量の順序で特徴量を選択することを考える。

その中で、特徴量と目的変数の 1:1 の関係に着目した単変量統計について考える。

### 相関係数

各特徴量と目的変数の相関係数を計算して絶対値の大きい順に選択する。  
線形以外の関係性を捉えることはできないので注意。

$$
\rho = \frac{\sum_i(x_i - x_\mu)(y_i - y_\mu)}{\sqrt{\sum_i(x_i - x_\mu)^2\sum_i(y_i - y_\mu)^2}}
$$

スピアマンの順位相関係数

### カイ二乗統計量

